In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
base_dir = '/content/imagedb_btsd.zip'

train_dir = os.path.join(base_dir, 'imagedb')
validation_dir = os.path.join(base_dir, 'imagedb_test')

In [16]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  #This layer creates a convolution kernel that is convolved 
  #with the layer inputo produce a tensor of outputs.
  tf.keras.layers.Conv2D(16, (5, 5), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  #34 classes in dataset
  tf.keras.layers.Dense(34, activation='softmax')
])

model.compile(optimizer = tf.keras.optimizers.RMSprop(),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 24, 24, 16)        416       
                                                                 
 dropout (Dropout)           (None, 24, 24, 16)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 16)       0         
 2D)                                                             
                                                                 
 flatten_7 (Flatten)         (None, 2304)              0         
                                                                 
 dense_14 (Dense)            (None, 100)               230500    
                                                                 
 dense_15 (Dense)            (None, 34)                3434      
                                                      

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
test_datagen  = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=100,
                                                    class_mode='categorical',
                                                    color_mode='grayscale',
                                                    target_size=(28,28))     

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=100,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(28,28))

Found 3056 images belonging to 34 classes.
Found 2149 images belonging to 34 classes.


In [28]:
import datetime

callbacks = []

logdir = os.path.join("/content/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
callbacks.append(tensorboard_callback)

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=30,
                              epochs=50,
                              validation_steps=15,
                              verbose=1,
                              callbacks=callbacks)

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.0145 - accuracy: 0.9959
Epoch 00001: val_loss improved from inf to 1.07050, saving model to best_weights.hdf5
30/30 [==============================] - 3s 85ms/step - loss: 0.0145 - accuracy: 0.9959 - val_loss: 1.0705 - val_accuracy: 0.9140
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.0482 - accuracy: 0.9922
Epoch 00002: val_loss improved from 1.07050 to 0.97381, saving model to best_weights.hdf5
30/30 [==============================] - 2s 78ms/step - loss: 0.0482 - accuracy: 0.9922 - val_loss: 0.9738 - val_accuracy: 0.9140
Epoch 3/50
30/30 [==============================] - ETA: 0s - loss: 0.0016 - accuracy: 0.9993
Epoch 00003: val_loss did not improve from 0.97381
30/30 [==============================] - 2s 78ms/step - loss: 0.0016 - accuracy: 0.9993 - val_loss: 0.9975 - val_accuracy: 0.9253
Epoch 4/50
30/30 [==============================] - ETA: 0s - loss: 0.0339 - accuracy: 0.9956
Epoch 0

In [29]:
test_datagen  = ImageDataGenerator()
test_dir = os.path.join(base_dir, 'imagedb_test')
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=10,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(28,28)) 
loss, acc = model.evaluate(test_generator)


Found 2149 images belonging to 34 classes.
215/215 [==============================] - 1s 5ms/step - loss: 0.8990 - accuracy: 0.9325
